<a href="https://colab.research.google.com/github/burglarhobbit/low-res-nmt/blob/master/train_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!nvidia-smi

Fri Apr 17 02:15:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from pathlib import Path
import os
from collections import Counter
import numpy as np
import time
from tqdm.notebook import tqdm

np.random.seed(8080)

data_path = "/content/drive/My Drive/IFT6759-PROJECT2/data/bpe"
# print(data_path)
# print(os.listdir(data_path))
# os.chdir("/content/drive/My Drive/Adv Projects in ML/")
!PYTHONPATH=$PYTHONPATH:"/content/drive/My\ Drive/IFT6759-PROJECT2/data/bpe"
import sys
sys.path.append(data_path)

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
from transformer import Transformer, CustomSchedule, create_masks

print("Tensorflow version " + tf.__version__)
subword="_default"
train_lang1="train/split_train%s_bpe.lang1"%subword
train_lang2="train/split_train%s_bpe.lang2"%subword
val_lang1="val/split_val%s_bpe.lang1"%subword
val_lang2="val/split_val%s_bpe.lang2"%subword
tf.random.set_seed(8080)
# make sure numpy seeded

Tensorflow version 2.2.0-rc3


In [5]:
# read data
def read_file(path):
  abs_path = os.path.join(data_path,path)
  with open(abs_path,"r") as f:
    data = f.read()
  return data

def read_file_and_process(path,limit_vocab=10000):
  data = read_file(path)
  data_new = data.replace("\n", " <eos> ").split()
  vocab = list(set(data_new))
  counter = Counter(data_new)
  counter.update({"<unk>":0}) # artificial count
  counter.update({"<start>":0}) # artificial count
  # most_common = counter.most_common(limit_vocab)
  lang_vocab = list(counter.keys())
  # lang_vocab = [i for i,j in most_common]
  word2id = {}
  id2word = {}
  # start enumerate from 1 so that 0 is reserved for padding seqs 
  # for i, (w,count) in enumerate(most_common, start=1):
  for i, w in enumerate(lang_vocab, start=1):
    word2id[w] = i
    id2word[i] = w
  return data, lang_vocab, word2id, id2word

english, english_vocab, english_word2id, english_id2word = read_file_and_process(train_lang1,limit_vocab=9000)
french, french_vocab, french_word2id, french_id2word = read_file_and_process(train_lang2,limit_vocab=10000)
english_val = read_file(val_lang1)
french_val = read_file(val_lang2)

print(len(english.split("\n")), english[:200])
print(len(french.split("\n")), french[:200])
print(len(english_val.split("\n")), english_val[:200])
print(len(french_val.split("\n")), french_val[:200])

len(english_word2id), len(english_id2word), len(french_word2id), len(french_id2word)

def transform_data(english_lang1, french_lang2):
  english_lines = english_lang1.split("\n")
  french_lines = french_lang2.split("\n")

  data_english = []
  data_french = []

  for line in english_lines:
    line2id = [english_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(english_word2id[word])
      except:
        line2id.append(english_word2id["<unk>"])
    line2id.append(english_word2id["<eos>"])
    data_english.append(line2id)

  for line in french_lines:
    line2id = [french_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(french_word2id[word])
      except:
        line2id.append(french_word2id["<unk>"])
    line2id.append(french_word2id["<eos>"])
    data_french.append(line2id)

  print(len(data_english), len(data_french))
  return data_english, data_french

data_english, data_french = transform_data(english, french)
data_english_val, data_french_val = transform_data(english_val, french_val)

english_id2word[54], len(data_english), len(data_french), len(data_english_val), len(data_french_val)

8800 as mr de cas@@ tro is not present mr le fol@@ l who is re@@ plac@@ ing mr de cas@@ tro has the flo@@ or
on the other hand if you 're visi@@ ting an under@@ developed country and 25 dollars bu@@ ys you
8800 Comme M. De C@@ as@@ tro est abs@@ ent , M. Le F@@ ol@@ l , qui le rem@@ place , a la parole .
D' un autre côté , si vous êtes dans un pays en voie de développement , où 25 dollars peuvent vous obteni
2200 what action does the council intend to take in the face of this seriously discrimin@@ atory attitude which r@@ uns contrary to the principles of the eu
where would you like to go next
if that were not
2200 Quelles mesures le Conseil comp@@ te@@ -@@ t@@ -il adopter face à cette attitude qui constitue une grave discrimination et est contraire aux principes sur lesquels l' Union européenne est fondée ?
Où 
8800 8800
2200 2200


('your', 8800, 8800, 2200, 2200)

In [0]:
np.savez("data_and_vocab.npz", data_english=data_english, data_french=data_french, data_english_val=data_english_val, data_french_val=data_french_val, 
         english_word2id=english_word2id, english_id2word=english_id2word, french_word2id=french_word2id,french_id2word=french_id2word)

In [51]:
BUFFER_SIZE = len(data_english)
BATCH_SIZE = 64
EPOCHS = 50
print("No. of batches: ", np.ceil(len(data_english)/BATCH_SIZE))

# transformer hyperparams
num_layers = 1
d_model = 1024+256
dff = 1024+256
num_heads = 8
input_vocab_size = len(english_vocab) + 1
target_vocab_size = len(french_vocab) + 1
dropout_rate = 0.4
pe_input = max(max([len(i) for i in data_english]),max([len(i) for i in data_english_val]))
pe_target = max(max([len(i) for i in data_french]),max([len(i) for i in data_french_val]))

# pe_input = 200
# pe_target = 230

No. of batches:  138.0


In [0]:
tensor_train = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(data_english, padding='post'),
    tf.keras.preprocessing.sequence.pad_sequences(data_french, padding='post')
)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=False)
tensor_val = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(data_english_val, padding='post'),
    tf.keras.preprocessing.sequence.pad_sequences(data_french_val, padding='post')
)).batch(BATCH_SIZE, drop_remainder=False)

In [52]:
transformer = Transformer(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, 
    input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, 
    pe_input=pe_input, pe_target=pe_target, rate=dropout_rate)

temp_input = tf.random.uniform((BATCH_SIZE, pe_input), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((BATCH_SIZE, pe_target), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

TensorShape([64, 152, 6944])

In [53]:
transformer.summary()

Model: "transformer_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_3 (Encoder)          multiple                  17573120  
_________________________________________________________________
decoder_3 (Decoder)          multiple                  25292800  
_________________________________________________________________
dense_99 (Dense)             multiple                  8895264   
Total params: 51,761,184
Trainable params: 51,761,184
Non-trainable params: 0
_________________________________________________________________


In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_loss = tf.keras.metrics.Mean(name='loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='val_accuracy')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [0]:
experiment_number = "6_bpe_%d_%d_%d_%d" %(num_layers,d_model,dff,num_heads)

checkpoint_path = "./checkpoints/train"+experiment_number

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager_best_acc = tf.train.CheckpointManager(ckpt, checkpoint_path+"_acc", max_to_keep=1)
ckpt_manager_best_loss = tf.train.CheckpointManager(ckpt, checkpoint_path+"_loss", max_to_keep=1)

writer_train = tf.summary.create_file_writer("log_dir/"+experiment_number+"_train")
writer_val = tf.summary.create_file_writer("log_dir/"+experiment_number+"_val")

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [0]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

@tf.function(input_signature=train_step_signature)
def val_step(inp, tar):
  
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  predictions, _ = transformer(inp, tar_inp, 
                                False, 
                                enc_padding_mask, 
                                combined_mask, 
                                dec_padding_mask)
  loss = loss_function(tar_real, predictions)
  
  val_loss(loss)
  val_accuracy(tar_real, predictions)


In [57]:
best_val_loss = np.inf
best_val_acc = 0

for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  val_loss.reset_states()
  val_accuracy.reset_states()
  
  for (batch, (inp, tar)) in tqdm(enumerate(tensor_train)):
    train_step(inp, tar)
    if batch % 50 == 0:
      print ('Epoch {} Batch {} Training Loss {:.4f} Accuracy {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
    
  print ('Epoch {} Training Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))
  print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

  with writer_train.as_default():
    tf.summary.scalar('train_loss', train_loss.result(), step=epoch)

  print("validating")
  for (batch, (inp, tar)) in enumerate(tensor_val):
    val_step(inp, tar)
  
  print ('Epoch {} Validation Loss {:.4f} Accuracy {:.4f}'.format(epoch + 1, 
                                            val_loss.result(), 
                                            val_accuracy.result()))
  if best_val_loss > val_loss.result():
    best_val_loss = val_loss.result()
    ckpt_save_path = ckpt_manager_best_loss.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
  elif best_val_acc < val_accuracy.result():
    best_val_acc = val_accuracy.result()
    ckpt_save_path = ckpt_manager_best_acc.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
  with writer_val.as_default():
    tf.summary.scalar('val_loss', val_loss.result(), step=epoch)

Epoch 1 Batch 0 Training Loss 9.0157 Accuracy 0.0000
Epoch 1 Batch 50 Training Loss 8.8914 Accuracy 0.0007
Epoch 1 Batch 100 Training Loss 8.5484 Accuracy 0.0049

Epoch 1 Training Loss 8.2852 Accuracy 0.0065
Time taken for 1 epoch: 67.49490213394165 secs

validating
Epoch 1 Validation Loss 7.1942 Accuracy 0.0146
Saving checkpoint for epoch 1 at ./checkpoints/train6_bpe_1_1280_1280_8_loss/ckpt-1


Epoch 2 Batch 0 Training Loss 7.3695 Accuracy 0.0122
Epoch 2 Batch 50 Training Loss 7.1518 Accuracy 0.0116
Epoch 2 Batch 100 Training Loss 7.0052 Accuracy 0.0125

Epoch 2 Training Loss 6.9235 Accuracy 0.0133
Time taken for 1 epoch: 64.23487186431885 secs

validating
Epoch 2 Validation Loss 6.6910 Accuracy 0.0197
Saving checkpoint for epoch 2 at ./checkpoints/train6_bpe_1_1280_1280_8_loss/ckpt-2


Epoch 3 Batch 0 Training Loss 6.6994 Accuracy 0.0163
Epoch 3 Batch 50 Training Loss 6.6209 Accuracy 0.0176
Epoch 3 Batch 100 Training Loss 6.5664 Accuracy 0.0185

Epoch 3 Training Loss 6.5237 Accuracy 0.0191
Time taken for 1 epoch: 64.01979184150696 secs

validating
Epoch 3 Validation Loss 6.4421 Accuracy 0.0269
Saving checkpoint for epoch 3 at ./checkpoints/train6_bpe_1_1280_1280_8_loss/ckpt-3


Epoch 4 Batch 0 Training Loss 6.3415 Accuracy 0.0217
Epoch 4 Batch 50 Training Loss 6.2908 Accuracy 0.0226
Epoch 4 Batch 100 Training Loss 6.2269 Accuracy 0.0235

Epoch 4 Training Loss 6.1818 Accuracy 0.0241
Time taken for 1 epoch: 64.09188151359558 secs

validating
Epoch 4 Validation Loss 6.1845 Accuracy 0.0314
Saving checkpoint for epoch 4 at ./checkpoints/train6_bpe_1_1280_1280_8_loss/ckpt-4


Epoch 5 Batch 0 Training Loss 6.1189 Accuracy 0.0232
Epoch 5 Batch 50 Training Loss 5.9320 Accuracy 0.0271
Epoch 5 Batch 100 Training Loss 5.8794 Accuracy 0.0276

Epoch 5 Training Loss 5.8405 Accuracy 0.0279
Time taken for 1 epoch: 64.13698363304138 secs

validating
Epoch 5 Validation Loss 5.8792 Accuracy 0.0344
Saving checkpoint for epoch 5 at ./checkpoints/train6_bpe_1_1280_1280_8_loss/ckpt-5


...

Epoch 39 Batch 0 Training Loss 0.6007 Accuracy 0.1491
Epoch 39 Batch 50 Training Loss 0.6474 Accuracy 0.1490
Epoch 39 Batch 100 Training Loss 0.6785 Accuracy 0.1469

Epoch 39 Training Loss 0.7016 Accuracy 0.1459
Time taken for 1 epoch: 63.709741830825806 secs

validating
Epoch 39 Validation Loss 5.7038 Accuracy 0.0780


Epoch 40 Batch 0 Training Loss 0.5836 Accuracy 0.1598
Epoch 40 Batch 50 Training Loss 0.6039 Accuracy 0.1499
Epoch 40 Batch 100 Training Loss 0.6370 Accuracy 0.1482

Epoch 40 Training Loss 0.6605 Accuracy 0.1477
Time taken for 1 epoch: 63.70503306388855 secs

validating
Epoch 40 Validation Loss 5.7630 Accuracy 0.0781


Epoch 41 Batch 0 Training Loss 0.5604 Accuracy 0.1340
Epoch 41 Batch 50 Training Loss 0.5775 Accuracy 0.1526
Epoch 41 Batch 100 Training Loss 0.6074 Accuracy 0.1505

Epoch 41 Training Loss 0.6283 Accuracy 0.1492
Time taken for 1 epoch: 63.910191774368286 secs

validating
Epoch 41 Validation Loss 5.8431 Accuracy 0.0782


Epoch 42 Batch 0 Training Loss 0.5504 Accuracy 0.1649
Epoch 42 Batch 50 Training Loss 0.5592 Accuracy 0.1535


KeyboardInterrupt: ignored

Evaluate best model

In [73]:
# load model

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager_best_acc.latest_checkpoint:
  ckpt.restore(ckpt_manager_best_acc.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [74]:
ckpt_manager_best_acc.checkpoints

['./checkpoints/train6_bpe_1_1280_1280_8_acc/ckpt-26']

In [75]:
transformer

In [76]:
val_loss.reset_states()
val_accuracy.reset_states()
  
for (batch, (inp, tar)) in tqdm(enumerate(tensor_val)):
  val_step(inp, tar)
  
print ('Validation Loss {:.4f} Accuracy {:.4f}'.format(
                                          val_loss.result(), 
                                          val_accuracy.result()))
  


Validation Loss 4.6306 Accuracy 0.0787


In [0]:
def generate_predictions(inp_sentences):

  if len(inp_sentences.get_shape())==1:
    encoder_input = tf.expand_dims(inp_sentences, 0)
    decoder_input = [french_word2id["<start>"]]
    output = tf.expand_dims(decoder_input, 0)

  else:
    encoder_input = inp_sentences
    decoder_input = [french_word2id["<start>"]]*inp_sentences.get_shape()[0]
    output = tf.expand_dims(decoder_input, -1)


  # encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # decoder_input = [french_word2id["<start>"]]
  # output = tf.expand_dims(decoder_input, 0)
  
  for i in range(pe_target):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # # return the result if all the seqs has the end token
    if tf.reduce_sum(tf.cast((tf.reduce_sum(tf.cast(output == french_word2id["<eos>"], tf.float32),axis=1)>0), tf.float32)) == inp.get_shape()[0]:
      return output, attention_weights
    
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

  # return tf.squeeze(output, axis=0), attention_weights
  return output, attention_weights

In [78]:
all_preds = []
for (batch_i, (inp, tar)) in tqdm(enumerate(tensor_val)):
  preds, attention = generate_predictions(inp)
  all_preds.append(preds)

In [79]:
translated_sentences = []

for k in tqdm(all_preds):
  for i in k:
    sentence_french = []
    for j in i.numpy()[1:]:
      if j==0 or j==french_word2id["<eos>"]:
        break
      sentence_french.append(french_id2word[j])

    sentence_french = " ".join(sentence_french)

    translated_sentences.append(sentence_french)

translated_sentences = "\n".join(translated_sentences)

with open("predictions.txt","w") as f:
  f.write(translated_sentences)

In [80]:
!sed -r 's/(@@ )|(@@ ?$)//g' predictions.txt > predictions_new.txt
!pip install sacrebleu

In [81]:
!python evaluator.py --input-file-path ./predictions_new.txt --target-file-path './drive/My Drive/IFT6759-PROJECT2/data/bpe/split_val_bpe.lang2' --do-not-run-model 

scores: [1.0, 6.1, 2.6, 5.8, 8.6, 4.8, 8.1, 10.7, 13.7, 3.7, 13.7, 8.2, 9.2, 1.6, 2.7, 4.3, 1.4, 7.9, 3.6, 16.5, 4.6, 15.1, 10.8, 8.4, 2.5, 11.2, 20.4, 0.5, 15.9, 10.6, 10.0, 7.3, 7.4, 2.6, 2.6, 9.4, 17.8, 1.7, 8.8, 19.7, 7.4, 5.8, 15.0, 5.4, 4.0, 1.0, 4.1, 3.7, 7.2, 11.3, 10.1, 22.8, 2.9, 12.6, 4.5, 5.1, 7.7, 3.0, 1.1, 1.0, 6.7, 2.8, 3.9, 2.9, 7.1, 4.6, 6.3, 1.3, 3.8, 5.8, 2.2, 5.9, 4.9, 10.4, 3.5, 3.9, 10.9, 10.2, 4.0, 3.0, 3.1, 5.8, 9.7, 9.0, 9.5, 4.7, 2.9, 13.8, 4.1, 4.4, 9.5, 5.3, 1.9, 19.9, 2.0, 9.2, 7.8, 5.8, 3.7, 4.1, 1.1, 13.1, 10.8, 8.9, 13.5, 8.4, 7.6, 3.0, 2.2, 10.9, 1.7, 4.8, 1.5, 14.8, 9.4, 6.2, 20.2, 9.1, 3.9, 11.2, 19.3, 9.7, 9.5, 15.9, 66.9, 8.9, 16.2, 3.2, 12.5, 21.0, 4.5, 7.1, 8.5, 9.5, 16.5, 2.6, 5.2, 4.5, 4.9, 3.5, 10.4, 5.1, 4.1, 15.9, 10.7, 1.7, 2.4, 47.4, 30.2, 5.5, 1.2, 10.1, 6.0, 4.1, 9.8, 9.1, 4.6, 15.3, 21.7, 7.3, 10.7, 25.2, 8.5, 17.1, 17.8, 5.0, 10.2, 3.6, 1.9, 7.3, 1.4, 21.1, 0.9, 4.1, 7.1, 25.1, 2.8, 4.1, 17.7, 12.6, 2.9, 0.9, 7.2, 6.8, 3.7, 22.1, 5.2, 1

Self-Training Monolingual Data Generation

In [0]:
amount_data_start = 100000
amount_data_end = 200000

In [0]:
with open(data_path/"unaligned_tokenized_rempunc.en","r") as f:
    english_monolingual = f.read().strip().lower()
print(len(english_monolingual.split("\n")), english_monolingual[:200])

474000 for the second phase of the trials we just had different sizes small medium large and extra - large it 's true
geng had been my host the previous january when i was the first us defense secretary to v


In [0]:
def transform_test_data(lang1, dict_word2id, amount_data_start=None,amount_data_end=None):
  lines = lang1.split("\n")
  if amount_data_start or amount_data_end:
    lines = lines[amount_data_start:amount_data_end]
  data = []

  for line in lines:
    line2id = [dict_word2id["<start>"]]
    for word in line.split():
      try:
        line2id.append(dict_word2id[word])
      except:
        line2id.append(dict_word2id["<unk>"])
    line2id.append(dict_word2id["<eos>"])
    data.append(line2id)

  return data

english_monolingual_data = transform_test_data(english_monolingual, english_word2id, amount_data_start, amount_data_end)
len(english_monolingual_data)

100000

In [0]:
max([len(i) for i in english_monolingual_data])

112

In [0]:
with open(data_path/"unaligned.fr","r") as f:
    french_monolingual = f.read().strip().lower()
print(len(french_monolingual.split("\n")), french_monolingual[:200])
french_monolingual_data = transform_test_data(french_monolingual, french_word2id)
print(len(french_monolingual_data))
max([len(i) for i in french_monolingual_data])

474000 nous n’aurions pas pu dégager d’accord sur un calendrier de conclusion de la cig sans l’engagement politique de mes collègues du conseil européen.
(de) madame la présidente, monsieur le commissaire, m
474000


220

In [0]:
pe_input = max([len(i) for i in english_monolingual_data])
pe_target = max([len(i) for i in french_monolingual_data])

In [0]:
tensor_test = tf.data.Dataset.from_tensor_slices((
    tf.keras.preprocessing.sequence.pad_sequences(english_monolingual_data, padding='post')
)).batch(BATCH_SIZE, drop_remainder=False)

In [0]:
transformer = Transformer(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, 
    input_vocab_size=input_vocab_size, target_vocab_size=target_vocab_size, 
    pe_input=pe_input, pe_target=pe_target, rate=dropout_rate)

In [0]:
experiment_number = "7_smaller_1_1024_"

checkpoint_path = "./checkpoints/train"+experiment_number

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [0]:
all_preds = []
for batch_i, inp in tqdm(enumerate(tensor_test.unbatch().batch(128)),total=len(english_monolingual_data) // 128 + 1):
  preds, attention = generate_predictions(inp)
  all_preds.append(preds)

In [0]:
translated_sentences = []

for k in tqdm(all_preds):
  for i in k:
    sentence_french = []
    for j in i.numpy()[1:]:
      if j==0 or j==french_word2id["<eos>"]:
        break
      sentence_french.append(french_id2word[j])

    sentence_french = " ".join(sentence_french)

    translated_sentences.append(sentence_french)

translated_sentences = "\n".join(translated_sentences)

with open("predictions_english_monolingual_"+str(amount_data_start)+"_"+str(amount_data_end)+".txt","w") as f:
  f.write(translated_sentences)